In [50]:
import numpy as np
import matplotlib.pyplot as plt
import difflib
import csv
import turicreate as tc
import geopy.distance
# import skfuzzy as fuzz
# import numpy_indexed as npi
# import pyamg

from turicreate import SFrame, SArray
from math import sin, cos, sqrt, atan2, radians
# from scipy.spatial import distance
# from astropy.table import Table
# from astropy.io.ascii import write
# from sklearn.decomposition import PCA
# from sklearn import metrics
# from sklearn.cluster import KMeans
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import pairwise_distances
# from sklearn.mixture import GaussianMixture
# from sklearn.mixture import BayesianGaussianMixture
# from scipy.stats import entropy
# from skcmeans.algorithms import CMeans
# from skcmeans.algorithms import Probabilistic
# from skcmeans.algorithms import Fuzzy
# from pcm.cmeans import pcm

# from neupy import algorithms, environment

# from hmmlearn import hmm

In [2]:
gps_cat = ['ts', 'sensorid', 'gpslat', 'gpslon']
gps_sen = ['ts', 'sensorid', 'mac']
csv_gps = ['mov_gps_1.csv', 'mov_gps_kobe.csv']
csv_sen = ['mov_sensing_1.csv', 'mov_sensing_kobe.csv']

In [3]:
s = '-39bgm'
result = ''.join(i for i in s if i.isdigit() or i == '-')
result

'-39'

In [219]:
def returnShorter(a, b):
    if len(a) > len(b):
        return b
    else:
        return a
    
def load_data(input_csv, gps):
    line = []
    for file in input_csv:
        with open(file) as csvfile:
            reader = csv.reader(csvfile, delimiter=' ')
            for row in reader:
                if ';' in row[1]:
                    row[1] = row[1].split(';')
                else:
                    row[0] = format_transform(row[0])
                    row[1] = row[1].split(',')
                if '' in row[1]:
                    row[1].remove('')
                row[1].insert(0, row[0])
                del(row[1][2:4])
                if(gps):
                    del(row[1][4:])
                else:
                    s = row[1][5]
                    result = ''.join(i for i in s if i.isdigit() or i == '-')
                    del(row[1][4:])
                    row[1].insert(len(row[1])-1, result)
                line.append(row[1])
    return line
    
def check_error(count, mac, date):
    amount = 0
    for item in sen_frame:
        if item['mac'] == mac and item['date'] == date:
            amount+=1
    if amount != count:
        print(mac, date)

def check_last_index(a, element):
    index = -1
    position = 0
    for item in a:
        if item == element:
            index = position
        position+=1
    return index

def time_convert(time):
    s_list = time.split(':')
    if len(s_list) == 3:
        return 3600*float(s_list[0]) + 60*float(s_list[1]) + float(s_list[2])
    else:
        return 3600*float(s_list[0]) + 60*float(s_list[1])

def time_difference(time1, time2):
    return time_convert(time2) - time_convert(time1)

def find(tag1, con1, tag2, con2):
    for item in data:
        if item[tag1] == con1 and item[tag2] == con2:
            print(item)

def find_distance(s_lat, s_lon, e_lat, e_lon):
    R = 6373.0

    lat1 = radians(float(s_lat))
    lon1 = radians(float(s_lon))
    lat2 = radians(float(e_lat))
    lon2 = radians(float(e_lon))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c * 1000

def compute_distance(set1, set2):
    cumulative = 0
    summation = 0
    for i, j in zip(set1, set2):
        cumulative += (i-j)**2
        if j != 0:
            summation += i/j
        else:
            summation += i/1
    return sqrt(cumulative)

def normalize(v):
    largest = v.max()
    if largest == 0: 
        return v
    return v / largest

# Compute the Silhouette Coefficient
def compute_Sil_socre(data, labels):
    score = metrics.silhouette_score(data, labels, metric='euclidean')
    return score
    
def format_transform(date):
    # temp[0]: Month temp[1]: Day temp[2]: Year
    temp = date.split('/')
    if len(temp[1]) == 2 and len(temp[0]) == 1:
        return '20' + temp[2] + '-0' + temp[0] + '-' + temp[1]
    elif len(temp[1]) == 1 and len(temp[0]) == 2:
        return '20' + temp[2] + '-' + temp[0] + '-0' + temp[1]
    elif len(temp[1]) == 2 and len(temp[0]) == 2:
        return '20' + temp[2] + '-' + temp[0] + '-' + temp[1]
    else:
        return '20' + temp[2] + '-0' + temp[0] + '-0' + temp[1]

def computePassen(labels):  
    count = 0
    for i in range(len(labels)):
        if labels[i] == 1:
            count+=1
    print(count)

def evaluate(a, b):
    score_mape = mape(a, b)
    score_mse = mse(a, b)
    score_smape = smape(a, b)
    print("Mean Abosulute Percentage Error: ", score_mape)
    print("Mean Square Error: ", score_mse)
    print("Symmetry Mean Abosulute Percentage Error: ", score_smape)

def mape(a, b): 
    mask = a != 0
    return (np.fabs(a - b)/a)[mask].mean()

def mse(a, b):
    return mean_squared_error(a, b)

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

def multiply(a, scalor):
    for i in range(len(a)):
        a[i] = int(round(a[i] * scalor))
    return a

def list_difference(list1, list2):
    return list(set(list1) - set(list2))

def setNone(result):
    result_none = [[0 for a in range(len(result))] for b in range(len(result))]
    for i in range(len(result)):
        for j in range(len(result)):
            if j < i:
                result_none[i][j] = None
            else:
                result_none[i][j] = result[i][j]
    return result_none

def Relu(result):
    for i in range(len(result)):
        result[i] = result[i].apply(lambda x: round(x) if x > 0 else 0)
    return result

def weight_validation(data, label):
    # The lower the better
    # Search for labels(clusters)
    indices = []
    for i in range(2):
        indices.append(np.where(label == i)[0])
    # indices has two clusters labeled 0 and 1
    # Calculate the intra-cluster weights(Win)
    intra_weight = 0
    for i in range(2):
        for j in indices[i]:
            for k in indices[i]:
                intra_weight += compute_distance(data[j], data[k])
    intra_weight = intra_weight/2
    
    # Compute the inter-cluster weights(Wout)
    inter_weight = 0
    for i in indices[0]:
        for j in indices[1]:
            inter_weight += compute_distance(data[i], data[j])
    
    # Compute the number of distinct intra-cluster edges(Nin)
    intra_num = 0
    for i in range(2):
        intra_num += len(indices[i]) * (len(indices[i]) - 1) / 2 
    
    # Compute the number of distinct inter-cluster edges(Nout)
    inter_num = len(indices[0]) * len(indices[1])
    
    # BetaCV: smaller the BetaCV ratio, the better the clustering.
    BetaCV = (intra_weight / intra_num) / (inter_weight / inter_num)
    # Nomalized Cut: The higher the normalized cut value the better.
    Norm_index = inter_weight / (intra_weight + inter_weight)
    
    # BetaCV measure
    return BetaCV, Norm_index

def Dunn_index(data, label):
    # The larger the better
    # Search for labels(clusters)
    indices = []
    Win = []
    Wout = []
    for i in range(2):
        indices.append(np.where(label == i)[0])
    # Construct a list which are all the intra-cluster distance
    for i in range(2):
        for j in indices[i]:
            for k in indices[i]:
                Win.append(compute_distance(data[j], data[k]))
    # Construct a list which are all the inter-cluster distance
    for i in indices[0]:
        for j in indices[1]:
            Wout.append(compute_distance(data[i], data[j]))
    # Find Dunn Index
    return min(Wout)/max(Win)

def DaviesBouldin(data, label):
    # Smaller is better
    n_cluster = len(np.bincount(label))
    cluster_k = [data[label == k] for k in range(n_cluster)]
    centroids = [np.mean(k, axis = 0) for k in cluster_k]
    variances = [np.mean([compute_distance(p, centroids[i]) for p in k]) for i, k in enumerate(cluster_k)]
    db = []

    for i in range(n_cluster):
        for j in range(n_cluster):
            if j != i:
                db.append((variances[i] + variances[j]) / compute_distance(centroids[i], centroids[j]))

    return(np.max(db) / n_cluster)

def othertrip(date, first, last, frame):
    for i in frame:
        if i['date'] == date and time_difference(i['start_time'], first) >= 0:
            return (time_difference(last, i['end_time']) >= 0)
        
'''
This method extract the travel information and
put them into a SFrame
'''

def get_labeled_data():
    input_csv = 'trip_detail_new.csv'
    date_list = []
    time_list = []
    passen_list = []
    stop_id = []
    trip_id = []
    hour = []
    minute = []
    last_stop = []
    weekday = []
    route = []
    lat = []
    lon = []
    with open(input_csv) as csvfile:
        reader = csv.reader(csvfile)
        count = 0
        for row in reader:
            if count != 0:
                date = format_transform(row[5].split(' ')[0])
                stop = row[1]
                trip = row[0]
                trip_id.append(trip)
                date_list.append(date)
                stop_id.append(stop)
                lat.append(row[10])
                lon.append(row[11])
                hour.append(int(row[5].split(' ')[1].split(':')[0]))
                minute.append(int(row[5].split(' ')[1].split(':')[1]))
                last_stop.append(row[9])
                weekday.append(row[7])
                route.append(int(row[8]))
                time = row[5].split(' ')[1]
                time_list.append(time)
                passen_list.append(int(row[6]))
            count+=1   
    stop_frame = SFrame({'date': date_list, 'time': time_list, 'passenger': passen_list, 
                           'stop_id': stop_id, 'trip_id': trip_id, 'hour':hour, 'minute':minute,
                        'last_stop':last_stop, 'weekday':weekday, 'route':route, 'lat': lat, 'lon': lon})
    return stop_frame
    

In [4]:
# Make a frame which contains the data for each trip
with open('trip_detail_new.csv') as csvfile:
    reader = csv.reader(csvfile)
    count = 0
    start_time = []
    end_time = []
    trip_list = []
    date_list = []
    start = ''
    end = ''
    for row in reader:
        if count != 0:
            trip = row[0]
            start = row[5].split(' ')[1]
            date = format_transform(row[5].split(' ')[0])
            if trip not in trip_list:
                trip_list.append(trip)
                start_time.append(start)
                date_list.append(date)
                if end != '':
                    end_time.append(end)
            end = start
        count+=1
    end_time.append(end)
time_frame = SFrame({'start_time': start_time, 'end_time':end_time, 
                     'trip_id': trip_list, 'date': date_list})

In [5]:
stop_frame = get_labeled_data()

In [6]:
gps_data = load_data(csv_gps, True)
sen_data = load_data(csv_sen, False)

In [69]:
# Create SFrame for gps data and sensing data
def create_frame(list_data, name, time_frame):
    list_date = []
    list_time = []
    if 'gps' in name:
        list_lat = []
        list_lon = []
        for data in list_data:
        # Iterate thourgh the whole list of gps_data and extract values accordingly
            list_date.append(data[0])
            list_time.append(data[1])
            list_lat.append(data[2])
            list_lon.append(data[3])
        return SFrame({'date':list_date, 'time':list_time, 'lat': list_lat, 'lon':list_lon})
    else:
        list_mac = []
        list_source = []
        list_strength = []
        for data in list_data:
            time = data[1]
            date = data[0]
            in_range = False
            for item in time_frame:
                if date == item['date'] and time_difference(item['start_time'], time) > -120 and time_difference(time, item['end_time']) > -240:
                    in_range = True
            # Only those data within the range of trip
            if in_range:
            # Iterate thourgh the whole list of gps_data and extract values accordingly
                list_date.append(data[0])
                list_time.append(data[1])
                list_mac.append(data[2])
                if data[4] == 'wifi':
                    list_strength.append(data[3])
                else:
                    list_strength.append(str(0))
                list_source.append(data[4])
        return SFrame({'date':list_date, 'time':list_time, 'mac':list_mac, 'source':list_source, 'strength': list_strength})
    

In [105]:
gps_frame = create_frame(gps_data, 'gps', time_frame)
sen_frame = create_frame(sen_data, 'sensing', time_frame)
# gps_frame = gps_frame.unique()
gps_frame = gps_frame.sort(['date', 'time'])
# sen_frame = sen_frame.unique()
sen_frame = sen_frame.sort(['date', 'time'])

In [106]:
# Change the format of the time
list_time = []
for i in range(len(sen_frame)):
    time = sen_frame[i]['time'].split(':')
    time[2] = round(float(time[2]))
    if time[2] >= 60:
        time[1] = int(time[1]) + 1
        time[2] -= 60
    if int(time[1]) >= 60:
        time[0] = int(time[0]) + 1
        time[1] -= 60
    time[0] = str(time[0])
    time[1] = str(time[1])
    time[2] = str(time[2])
    if len(time[2]) == 1:
        time[2] = '0' + time[2]
    if len(time[1]) == 1:
        time[1] = '0' + time[1]
    if len(time[0]) == 1:
        time[0] = '0' + time[0]
    time = ''+time[0]+':'+time[1]+':'+str(time[2])
    list_time.append(time)
temp = SFrame({'time': list_time})
sen_frame = sen_frame.remove_columns(['time'])
sen_frame = sen_frame.add_columns(temp)

In [107]:
# Associate the GPS location with each MAC according to the time
list_lon = []
list_lat = []
date_list = []
for element in sen_frame:
    date = element['date']
    time = element['time']
    lon = ''
    lat = ''
    m = 99999999
    for i in gps_frame:
        if i['date'] == date and abs(time_difference(time, i['time'])) <= m:
            lat = i['lat']
            lon = i['lon']
            m = abs(time_difference(time, i['time']))
    list_lon.append(lon)
    list_lat.append(lat)
temp = SFrame({'lon': list_lon, 'lat': list_lat})
sen_frame = sen_frame.add_columns(temp)  


KeyboardInterrupt: 

In [ ]:
sla = '47.65677863'
slo = 
find_distance(47.65677863, )

In [ ]:
trip_id = []
for i in sen_frame:
    date = i['date']
    time = i['time']
    tripid = str(0)
    for j in time_frame:
        start_time = j['start_time']
        end_time = j['end_time']
        inRange = (time_difference(start_time, time) >= 0 and time_difference(time, end_time) >= 0)
        if j['date'] == date and inRange:
            tripid = j['trip_id']
    trip_id.append(tripid)
    
temp = SFrame({'trip_id': trip_id})
sen_frame = sen_frame.add_columns(temp)

In [13]:
s = []
for i in sen_frame:
    s.append(int(i['strength']))
p = np.percentile(s, 20)
p

-63.0

In [27]:
sf = SFrame.read_csv('clusterFuzzy_RSSI.csv')
sf

Finished parsing file /Users/hantaoliu/Documents/OD/clusterFuzzy_RSSI.csv

Parsing completed. Parsed 100 lines in 0.043785 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,float,float,float,float,int,float,str,str,str,str,float,float,float,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/hantaoliu/Documents/OD/clusterFuzzy_RSSI.csv

Parsing completed. Parsed 5064 lines in 0.019047 secs.

detectionNum,distance,speedMax,distanceToEnd,distanceToStart,duration,speedAvg,date
3,10106.29237,6.936183455,10.46295131,31.74115829,1767,5.719463708,2018-03-06
481,10652.13895,31.34644542,54.79991485,31.74115829,2651,4.018158791,2018-03-06
73,6622.839611,18.47044277,226.6639945,31.74115829,1273,5.202544864,2018-03-06
13,16131.96832,6.926942375,97.1824588,31.74115829,2602,6.199834098,2018-03-06
58,14698.16628,9.474226212,27.11112899,31.74115829,2188,6.717626269,2018-03-06
60,17423.43879,26.09527297,18.32649848,31.74115829,504,34.57031506,2018-03-06
9,6695.774204,33.96569769,172.2501309,31.74115829,1055,6.346705407,2018-03-06
6,526.3401448,66.30197935,38.00310863,31.74115829,25,21.05360579,2018-03-06
6,2786.89674,17.97232839,674.4758407,31.74115829,442,6.305196245,2018-03-06
4,2752.623568,10.93084318,391.4194955,136.0761018,451,6.1033782,2018-03-06


In [24]:
sen_frame.export_csv('sen_frame.csv')

In [99]:
list_avg = []
list_max = []
list_c = []
for i in data:
    avg = 0
    m = -9999999
    count = 0
    mac = i['mac']
    date = i['date']
    st = i['start_time']
    et = i['end_time']
    for j in sen_frame:
        if mac == j['mac'] and date == j['date'] and time_difference(st, j['time']) >= 0 and time_difference(j['time'], et) >= 0:
            avg += int(j['strength'])
            count += 1
            if int(j['strength']) > m:
                m = int(j['strength'])
    if count != 0:
        list_avg.append(avg / count)
        list_max.append(m)
        list_c.append(count)
    else:
        list_avg.append(0)
        list_max.append(0)
        list_c.append(0)
data = data.add_columns(SFrame({'avg_RSSI': list_avg, 'max_RSSI': list_max, 'number': list_c}))
            

In [35]:
sf.export_csv('clusterFuzzy_RSSI1.csv')

In [82]:
'''
This step check each MAC address and see if it is in the list, or
if it is on the other trip. For every MAC it store, it is going to
search for the rest which are in the same trip. Count them up and 
store the first and last detection time for each MAC.
'''

list_mac = []
list_start = []
list_end = []
list_count = []
list_date = []
list_first_lon = []
list_first_lat = []
list_last_lon = []
list_last_lat = []
list_trip = []
trip_id = []
list_source = []
list_stren = []
for i in time_frame:
    date = i['date']
    start_time = i['start_time']
    end_time = i['end_time']
    trip = i['trip_id']
    sub_mac = []
    sub_start = []
    sub_end = []
    sub_count = []
    sub_date = []
    sub_first_lon = []
    sub_first_lat = []
    sub_last_lon = []
    sub_last_lat = []
    sub_trip = []
    sub_source = []
    sub_stren = []
    for j in sen_frame:
        mac = j['mac']
        time = j['time']
        inRange = (time_difference(start_time, time) >= 0 and time_difference(time, end_time) >= 0)
        lon = j['lon']
        lat = j['lat']
        source = j['source']
        strength = j['strength']
        if date == j['date'] and inRange: 
            if mac not in sub_mac:
                sub_mac.append(mac)
                sub_count.append(1)
                sub_start.append(time)
                sub_end.append(time)
                sub_date.append(date)
                sub_first_lon.append(lon)
                sub_first_lat.append(lat)
                sub_last_lon.append(lon)
                sub_last_lat.append(lat)
                sub_trip.append(trip)
                sub_source.append(source)
                sub_stren.append(strength)
            else:
                index = check_last_index(sub_mac, mac)
                sub_count[index] += 1 
                sub_end[index] = time
                sub_last_lon[index] = lon
                sub_last_lat[index] = lat
                if int(strength) > int(sub_stren[index]):
                    sub_stren[index] = strength
    list_mac += sub_mac
    list_start += sub_start
    list_end += sub_end
    list_count += sub_count
    list_date += sub_date
    list_first_lon += sub_first_lon
    list_first_lat += sub_first_lat
    list_last_lon += sub_last_lon
    list_last_lat += sub_last_lat
    list_trip += sub_trip
    list_source += sub_source
    list_stren += sub_stren
    
data = SFrame({'mac':list_mac, 'date':list_date, 'start_time':list_start, 
               'end_time':list_end, 'count':list_count, 'start_lat': list_first_lat,
               'start_lon': list_first_lon, 'end_lat': list_last_lat, 'end_lon': list_last_lon,
               'trip_id': list_trip, 'source': list_source, 'strength': list_stren})


In [93]:
# Going through the data for each MAC compute the distance
# between it and the adjacent MAC which is the same as it
# This step will compute the time and location difference 
# between two detection of a same MAC, and compute the 
# distance as well as the speed between two detection.
# It will create two list which stores the total distance
# by adding up every piece of location difference together
# It will also store the maximum speed for every MAC address
list_distance = []
list_max_speed = []
for item in data:
    iteration = item['count']
    MAC = item['mac']
    distance = 0
    max_speed = 0
    count = 0
    prev_lon = 0
    prev_lat = 0
    prev_time = 0
    st = item['start_time']
    et = item['end_time']
    for j in gps_frame:
        if j['date'] == item['date'] and time_difference(st, j['time']) >= 0 and time_difference(j['time'], et) >= 0:
            if count == 0:
                prev_lat = float(j['lat'])
                prev_lon = float(j['lon'])
                prev_time = j['time']
            else:
                now_lat = float(j['lat'])
                now_lon = float(j['lon'])
                coords_1 = (prev_lat, prev_lon)
                coords_2 = (now_lat, now_lon)
                now_distance = geopy.distance.geodesic(coords_1, coords_2).m
                distance += now_distance
                time = time_difference(prev_time, j['time'])
                if time != 0 and max_speed < now_distance/time:
                    max_speed = now_distance/time
                prev_lon = now_lon
                prev_lat = now_lat
            count += 1
            
    list_distance.append(distance)
    list_max_speed.append(max_speed)
    
temp = SFrame({'distance': list_distance, 'max_speed': list_max_speed})
data = data.add_columns(temp)

(47.62448045, -122.4053699)
(47.62463256, -122.4051861)
now_distance 21.83761902856412
1.0
19:48:32
21.83761902856412

(47.62463256, -122.4051861)
(47.62463256, -122.4051861)
now_distance 0.0
2.0
19:48:33
21.83761902856412

(47.62463256, -122.4051861)
(47.6247444, -122.405121)
now_distance 13.362851219605405
2.0
19:48:33
21.83761902856412

(47.6247444, -122.405121)
(47.6247444, -122.405121)
now_distance 0.0
3.0
19:48:34
21.83761902856412

(47.6247444, -122.405121)
(47.62482464, -122.4051081)
now_distance 8.973863396345738
3.0
19:48:34
21.83761902856412

(47.62482464, -122.4051081)
(47.62482464, -122.4051081)
now_distance 0.0
4.0
19:48:35
21.83761902856412

(47.62482464, -122.4051081)
(47.62482749, -122.4051135)
now_distance 0.514931312812042
4.0
19:48:35
21.83761902856412

(47.62482749, -122.4051135)
(47.62484962, -122.4051165)
now_distance 2.4707915196241292
5.0
19:48:36
21.83761902856412

(47.62484962, -122.4051165)
(47.62482749, -122.4051135)
now_distance 2.4707915196241292
5.0
19:4

(47.65493467, -122.3753207)
(47.65500417, -122.3752868)
now_distance 8.136082564586673
151.0
19:51:02
79.70079818673835

(47.65500417, -122.3752868)
(47.65507317, -122.3752182)
now_distance 9.241826264796705
152.0
19:51:03
79.70079818673835

(47.65507317, -122.3752182)
(47.65514098, -122.375138)
now_distance 9.65087505437897
152.0
19:51:03
79.70079818673835

(47.65514098, -122.375138)
(47.655141, -122.375138)
now_distance 0.0022236725752933432
153.0
19:51:04
79.70079818673835

(47.655141, -122.375138)
(47.65521517, -122.3750368)
now_distance 11.216021449079923
153.0
19:51:04
79.70079818673835

(47.65521517, -122.3750368)
(47.65521517, -122.3750368)
now_distance 0.0
154.0
19:51:05
79.70079818673835

(47.65521517, -122.3750368)
(47.65524967, -122.3749328)
now_distance 8.703472124911746
154.0
19:51:05
79.70079818673835

(47.65524967, -122.3749328)
(47.65524967, -122.3749328)
now_distance 0.0
155.0
19:51:06
79.70079818673835

(47.65524967, -122.3749328)
(47.65525233, -122.3748792)
now_dist

79.70079818673835

(47.651949, -122.3664763)
(47.65191217, -122.3663145)
now_distance 12.826586211490932
240.0
19:52:31
79.70079818673835

(47.65191217, -122.3663145)
(47.65191217, -122.3663145)
now_distance 0.0
241.0
19:52:32
79.70079818673835

(47.65191217, -122.3663145)
(47.651851, -122.3661497)
now_distance 14.125802771865166
241.0
19:52:32
79.70079818673835

(47.651851, -122.3661497)
(47.651851, -122.3661497)
now_distance 0.0
242.0
19:52:33
79.70079818673835

(47.651851, -122.3661497)
(47.65180467, -122.3659732)
now_distance 14.225162978706317
242.0
19:52:33
79.70079818673835

(47.65180467, -122.3659732)
(47.65180467, -122.3659732)
now_distance 0.0
243.0
19:52:34
79.70079818673835

(47.65180467, -122.3659732)
(47.65175583, -122.365787)
now_distance 15.00549807297416
243.0
19:52:34
79.70079818673835

(47.65175583, -122.365787)
(47.65175583, -122.365787)
now_distance 0.0
244.0
19:52:35
79.70079818673835

(47.65175583, -122.365787)
(47.6517065, -122.3656092)
now_distance 14.439634356

19:54:42
79.70079818673835

(47.647017, -122.3515195)
(47.646954, -122.3513812)
now_distance 12.531344317117474
372.0
19:54:43
79.70079818673835

(47.646954, -122.3513812)
(47.646895, -122.3512505)
now_distance 11.80939006597677
373.0
19:54:44
79.70079818673835

(47.646895, -122.3512505)
(47.64682567, -122.3511115)
now_distance 12.980202970557572
374.0
19:54:45
79.70079818673835

(47.64682567, -122.3511115)
(47.64677117, -122.3509775)
now_distance 11.750717530310615
374.0
19:54:45
79.70079818673835

(47.64677117, -122.3509775)
(47.6467135, -122.350859)
now_distance 10.971870569421595
375.0
19:54:46
79.70079818673835

(47.6467135, -122.350859)
(47.64677117, -122.3509775)
now_distance 10.971870569421595
375.0
19:54:46
79.70079818673835

(47.64677117, -122.3509775)
(47.6467135, -122.350859)
now_distance 10.971870569421595
376.0
19:54:47
79.70079818673835

(47.6467135, -122.350859)
(47.64665567, -122.3507675)
now_distance 9.412927392561436
376.0
19:54:47
79.70079818673835

(47.64665567, -1

19:56:29
79.70079818673835

(47.64922067, -122.3498928)
(47.64923367, -122.3498828)
now_distance 1.6289837332612134
479.0
19:56:30
79.70079818673835

(47.64923367, -122.3498828)
(47.64924533, -122.349876)
now_distance 1.3934320497892385
479.0
19:56:30
79.70079818673835

(47.64924533, -122.349876)
(47.64924533, -122.349876)
now_distance 0.0
480.0
19:56:31
79.70079818673835

(47.64924533, -122.349876)
(47.6492525, -122.3498647)
now_distance 1.1645811367777537
480.0
19:56:31
79.70079818673835

(47.6492525, -122.3498647)
(47.64925967, -122.349854)
now_distance 1.132140385776573
481.0
19:56:32
79.70079818673835

(47.64925967, -122.349854)
(47.6492525, -122.3498647)
now_distance 1.132140385776573
481.0
19:56:32
79.70079818673835

(47.6492525, -122.3498647)
(47.64926017, -122.3498515)
now_distance 1.3079464966134133
482.0
19:56:33
79.70079818673835

(47.64926017, -122.3498515)
(47.64925967, -122.349854)
now_distance 0.19587867753784668
482.0
19:56:33
79.70079818673835

(47.64925967, -122.3498

584.0
19:58:15
79.70079818673835

(47.6501225, -122.3500982)
(47.6501345, -122.3500727)
now_distance 2.3345873858506296
584.0
19:58:15
79.70079818673835

(47.6501345, -122.3500727)
(47.6501345, -122.3500727)
now_distance 0.0
585.0
19:58:16
79.70079818673835

(47.6501345, -122.3500727)
(47.65014033, -122.350026)
now_distance 3.567876898910773
585.0
19:58:16
79.70079818673835

(47.65014033, -122.350026)
(47.65015233, -122.349959)
now_distance 5.207429184738544
586.0
19:58:17
79.70079818673835

(47.65015233, -122.349959)
(47.65014033, -122.350026)
now_distance 5.207429184738544
586.0
19:58:17
79.70079818673835

(47.65014033, -122.350026)
(47.65016233, -122.3498188)
now_distance 15.757628628290856
587.0
19:58:18
79.70079818673835

(47.65016233, -122.3498188)
(47.65015233, -122.349959)
now_distance 10.591531753307933
587.0
19:58:18
79.70079818673835

(47.65015233, -122.349959)
(47.65016233, -122.3498188)
now_distance 10.591531753307933
588.0
19:58:19
79.70079818673835

(47.65016233, -122.34

763.0
20:01:14
79.70079818673835

(47.65305567, -122.336111)
(47.6529395, -122.3361127)
now_distance 12.91682656382358
763.0
20:01:14
79.70079818673835

(47.6529395, -122.3361127)
(47.65317317, -122.336111)
now_distance 25.9805786925521
764.0
20:01:15
79.70079818673835

(47.65317317, -122.336111)
(47.65305567, -122.336111)
now_distance 13.064069606556016
764.0
20:01:15
79.70079818673835

(47.65305567, -122.336111)
(47.65317317, -122.336111)
now_distance 13.064069606556016
765.0
20:01:16
79.70079818673835

(47.65317317, -122.336111)
(47.65328033, -122.3361173)
now_distance 11.923828188348963
765.0
20:01:16
79.70079818673835

(47.65328033, -122.3361173)
(47.653389, -122.3361205)
now_distance 12.084710435420547
766.0
20:01:17
79.70079818673835

(47.653389, -122.3361205)
(47.65328033, -122.3361173)
now_distance 12.084710435420547
766.0
20:01:17
79.70079818673835

(47.65328033, -122.3361173)
(47.65350117, -122.3361252)
now_distance 24.56095233770274
767.0
20:01:18
79.70079818673835

(47.653

In [18]:
# list_detectPerMile = []
# for i in data:
#     detectPerMile = 0
#     miles = i['distance']/1609.34
#     count = i['count']
#     if not miles == 0:
#         detectPerMile = count / miles
#     list_detectPerMile.append(detectPerMile)
# data = data.add_columns(SFrame({'DPM': list_detectPerMile}))

In [23]:
# thMin = 3 * 60
# thStren = p
# thDis = 274.32
# thDPM = 10
# list_passen = []
# for i in data:
#     ifMin = False
#     ifStren = False
#     ifDis = False
#     ifDPM = False
#     m = i['duration']
#     stren = i['strength']
#     dis = i['distance']
#     dpm = i['DPM']
#     if m >= thMin:
#         ifMin = True
#     if int(stren) >= int(thStren):
#         ifStren = True
#     if dis >= thDis:
#         ifDis = True
#     if dpm >= thDPM:
#         ifDPM = True
    
#     if ifMin and ifStren and ifDis and ifDPM:
#         list_passen.append(1)
#     else:
#         list_passen.append(0)

In [24]:
data = data.add_columns(SFrame({'label': list_passen}))

In [ ]:
47.755611-122.233551

In [102]:
'''
This step intend to calculate the distance between closed stop 
when each map is detected and disappear
'''
stop_frame = get_labeled_data()
least_start_distance = []
least_end_distance = []

for i in data:
    trip = i['trip_id']
    start_lat = i['start_lat']
    start_lon = i['start_lon']
    end_lat = i['end_lat']
    end_lon = i['end_lon']
    min_first = 9999999999
    min_end = 9999999999
    if i['mac'] == 'd0:13:fd:52:ca:a3':
        for j in stop_frame:
            if j['trip_id'] == trip:
                coords_1 = (start_lat, start_lon)
                coords_2 = (j['lat'], j['lon'])
                coords_3 = (end_lat, end_lon)
                first_distance = geopy.distance.geodesic(coords_1, coords_2).m
                end_distance = geopy.distance.geodesic(coords_3, coords_2).m
                print(coords_2)
                print(coords_3)
                print(min_end)
                if first_distance < min_first:
                    min_first = first_distance
                if end_distance < min_end:
                    min_end = end_distance
                
#     least_start_distance.append(min_first)
#     least_end_distance.append(min_end)

# temp = SFrame({'least_distance_start': least_start_distance, 'least_distance_end': least_end_distance})
# data = data.add_columns(temp)
        

('47.652329', '-122.306099')
('47.751844833', '-122.211221833')
9999999999
('47.657257', '-122.304573')
('47.751844833', '-122.211221833')
13158.005351682277
('47.666599', '-122.300514')
('47.751844833', '-122.211221833')
12636.683459113803
('47.668735', '-122.300529')
('47.751844833', '-122.211221833')
11607.49561316213
('47.676003', '-122.300545')
('47.751844833', '-122.211221833')
11414.979584271667
('47.683147', '-122.300667')
('47.751844833', '-122.211221833')
10771.737620038362
('47.686481', '-122.300743')
('47.751844833', '-122.211221833')
10167.699230023964
('47.691326', '-122.301987')
('47.751844833', '-122.211221833')
9895.982382229397
('47.695759', '-122.305252')
('47.751844833', '-122.211221833')
9573.353998365466
('47.705028', '-122.301521')
('47.751844833', '-122.211221833')
9415.47046669659
('47.70858', '-122.301208')
('47.751844833', '-122.211221833')
8542.898890908333
('47.711124', '-122.299957')
('47.751844833', '-122.211221833')
8288.846831172998
('47.715919', '-122.

In [64]:
coords_1 = (47.76304, -122.1948822)
coords_2 = (47.719047, -122.2954718)
geopy.distance.geodesic(coords_1, coords_2).m

8990.984795818975

In [57]:
data = data.remove_columns(['distance', 'least_distance_end', 'min_distance_start'])

In [96]:
# Duration
list_duration = []
for element in data:
    duration = time_difference(element['start_time'], element['end_time'])
    list_duration.append(duration)
temp = SFrame({'duration':list_duration})
data = data.add_columns(temp)

# Speed
list_speed = []
for element in data:
    if element['duration'] == 0:
        list_speed.append(0)
    else:
        speed = element['distance'] / element['duration']
        list_speed.append(speed)
temp = SFrame({'avg_speed':list_speed})
data = data.add_columns(temp)

ValueError: Column 'duration' already exists in current SFrame

In [100]:
data.export_csv('data.csv')

In [ ]:
for j in time_frame:
    wifi_count = 0
    bluetooth_count = 0
    for i in sen_frame:
        if i['trip_id'] == j['trip_id'] and i['source'] == 'wifi':
            wifi_count += 1
        if i['trip_id'] == j['trip_id'] and i['source'] == 'bluetooth':
            bluetooth_count += 1
    print(j['trip_id'])
    print('wifi: ', wifi_count)
    print('bluetooth: ', bluetooth_count)
    print()

In [74]:
myFile = open('sen_frame.csv', 'w')  
with myFile:  
    writer = csv.writer(myFile)
    writer.writerows(sen_frame.to_dataframe().values)

In [75]:
feature_data = data.remove_columns(['date', 'start_time', 'end_time', 'mac'])

In [76]:
# Normalized the features
features = ['duration', 'count', 'distance', 'avg_speed', 'max_speed']
for f in features:
    feature_data[f] = normalize(feature_data[f])

In [59]:
# Make the panda frame into ndarray
input_data = feature_data.to_dataframe().values

In [210]:
pca = PCA(n_components=2).fit_transform(input_data)

# Kmeans
pred_k = KMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300, 
                tol=0.0001, precompute_distances='auto', verbose=0, 
                random_state=None, copy_x=True, n_jobs=None, algorithm='auto')
pred_kmeans = pred_k.fit_predict(input_data)
print('Silhouette Socre(closer to 1): ', compute_Sil_socre(input_data, pred_kmeans))

# # BayesianGaussianMixture
pred_b = BayesianGaussianMixture(n_components=2)
pred_BGM = pred_b.fit_predict(input_data)
# print('BayesianGaussianMixture Measurement')
# print('Silhouette Socre(closer to 1): ', compute_Sil_socre(input_data, pred_BGM))
# beataCV, normalizedCut = weight_validation(input_data, pred_BGM)
# print('Beta CV Measure(lower): ', beataCV)
# print('Normalized cut(higher): ', normalizedCut)
# print('Dunn index(higher): ', Dunn_index(input_data, pred_BGM))
# print('Davies-Bouldin Index(lower): ', DaviesBouldin(input_data, pred_BGM))

# Get the result from Gussian Mixture
pred_g = GaussianMixture(n_components = 2)
pred_Gaussian = pred_g.fit_predict(input_data)
# print('Gussian Mixture Measurement')
# print('Silhouette Socre(closer to 1): ', compute_Sil_socre(input_data, pred_Gaussian))
# beataCV, normalizedCut = weight_validation(input_data, pred_Gaussian)
# print('Beta CV Measure(lower): ', beataCV)
# print('Normalized cut(higher): ', normalizedCut)
# print('Dunn index(higher): ', Dunn_index(input_data, pred_Gaussian))
# print('Davies-Bouldin Index(lower): ', DaviesBouldin(input_data, pred_Gaussian))

# PCM
v, v0, u, u0,_,_ = pcm(input_data, 2, 2,0.005,10000)
pred_PCM = np.argmax(v, axis=0)
# print('Probabilistic CM Measurement')
# print('Silhouette Socre(closer to 1): ', compute_Sil_socre(input_data, pred_PCM))
# beataCV, normalizedCut = weight_validation(input_data, pred_PCM)
# print('Beta CV Measure(lower): ', beataCV)
# print('Normalized cut(higher): ', normalizedCut)
# print('Dunn index(higher): ', Dunn_index(input_data, pred_PCM))
# print('Davies-Bouldin Index(lower): ', DaviesBouldin(input_data, pred_PCM))

# # Try fuzzy c-means clustering
cntr, u_orig, _, _, _, _, _ = fuzz.cluster.cmeans(input_data, 2, 2, error=0.0005, maxiter=100000)
pred_Fuzzy = np.argmax(cntr, axis=0)
# print('Fuzzy Measurement')
# print('Silhouette Socre(closer to 1): ', compute_Sil_socre(input_data, pred_Fuzzy))
# beataCV, normalizedCut = weight_validation(input_data, pred_Fuzzy)
# print('Beta CV Measure(lower): ', beataCV)
# print('Normalized cut(higher): ', normalizedCut)
# print('Dunn index(higher): ', Dunn_index(input_data, pred_Fuzzy))
# print('Davies-Bouldin Index(lower): ', DaviesBouldin(input_data, pred_Fuzzy))

# SOFM
sofm = algorithms.SOFM(n_inputs=5, n_outputs=2,step=1,distance='cos')
sofm.train(input_data, epochs=100)
pred_SOFM = sofm.predict(input_data)
pred_SOFM = np.argmax(pred_SOFM, axis = 1)
# print('SOFM Measurement')
# print('Silhouette Socre(closer to 1): ', compute_Sil_socre(input_data, pred_SOFM))
# beataCV, normalizedCut = weight_validation(input_data, pred_SOFM)
# print('Beta CV Measure(lower): ', beataCV)
# print('Normalized cut(higher): ', normalizedCut)
# print('Dunn index(higher): ', Dunn_index(input_data, pred_SOFM))
# print('Davies-Bouldin Index(lower): ', DaviesBouldin(input_data, pred_SOFM))

Silhouette Socre(closer to 1):  0.9681520041178342


In [112]:
'''
This method computes the real time model validation score
and do the visualization

parameters: model {'fuzzy', 'GMM', 'BGM', 'SOFM', 'PCM'} 
            'fuzzy' is Fuzzy clustering
            'GMM' is Gaussian Mixture Model
            'PCM' is Probablistic c-means clustering
            'SOFM' is Self-Organzing Map
            'BGM' is Bayesian Gaussian Mixture Model
'''
def real_time_validation(model, data, i):
    xaxis = []
    betacv = []
    NormalizedCut = []
    DunnIndex = []
    DBIndex = []
    sil_score = []
    for i in range(len(data)):
        if i != 0 and i % 500 == 0:
            input_data = data[0:i+1]
            if model == 'fuzzy':
                cntr, u_orig, _, _, _, _, _ = fuzz.cluster.cmeans(input_data, 2, 2, error=0.0005, maxiter=100000)
                pred = np.argmax(cntr, axis=0)
            elif model == 'BGM':
                pred_b = BayesianGaussianMixture(n_components=2)
                pred = pred_b.fit_predict(input_data)
            elif model == 'GMM':
                pred_g = GaussianMixture(n_components = 2)
                pred = pred_g.fit_predict(input_data)
            elif model == 'SOFM':
                sofm = algorithms.SOFM(n_inputs=5, n_outputs=2,step=1,distance='cos')
                sofm.train(input_data, epochs=100)
                pred = sofm.predict(input_data)
                pred = np.argmax(pred, axis = 1)
            elif model == 'PCM':
                v, v0, u, u0,_,_ = pcm(input_data, 2, 2,0.005,10000)
                pred = np.argmax(v, axis=0)
            # Compute the result for every iteration
            sil = compute_Sil_socre(input_data, pred)
            beataCV, nc = weight_validation(input_data, pred)
            dunn = Dunn_index(input_data, pred)
            db = DaviesBouldin(input_data, pred)
            xaxis.append(i)
            sil_score.append(sil)
            betacv.append(beataCV)
            NormalizedCut.append(nc)
            DunnIndex.append(dunn)
            DBIndex.append(db)
        
    plt.figure(i+1)
    plt.plot(xaxis, betacv, 'r-', label='beta cv')
    plt.figure(i+2)
    plt.plot(xaxis, NormalizedCut, 'r-', label='normalized cut')
    plt.figure(i+3)
    plt.plot(xaxis, DunnIndex, 'r-', label='Dunn index')
    plt.figure(i+4)
    plt.plot(xaxis, DBIndex, 'r-', label = 'Davies Bouldin index')
    plt.figure(i+5)
    plt.plot(xaxis, sil_score, 'r-', label='Silhouette score')
    plt.legend()
    plt.show()
    return sil_score, betacv, NormalizedCut, DunnIndex, DBIndex
    

In [ ]:
# # Fuzzy
# print('Fuzzy')
# sil_score_f, betacv_f, NormalizedCut_f, DunnIndex_f, DBIndex_f = real_time_validation('fuzzy', input_data, 0)

# # Gaussian
# print('Gaussian')
# sil_score_g, betacv_g, NormalizedCut_g, DunnIndex_g, DBIndex_g = real_time_validation('GMM', input_data, 5)


In [ ]:
# # PCM
# print('PCM')
# sil_score_p, betacv_p, NormalizedCut_p, DunnIndex_p, DBIndex_p = real_time_validation('PCM', input_data, 10)

# # BGM
# print('BGM')
# sil_score_b, betacv_b, NormalizedCut_b, DunnIndex_b, DBIndex_b = real_time_validation('BGM', input_data, 15)

# SOFM
# print('SOFM')
# sil_score_s, betacv_s, NormalizedCut_s, DunnIndex_s, DBIndex_s = real_time_validation('SOFM', input_data, 20)


In [211]:
list_passen = list(data['mac'])

In [79]:
'''
This method takes in the predicted label
It returns the converted labels if number
of passenger is greater than number of noise
'''
def convert_label(pred):
    if (pred==0).sum() < (pred==1).sum():
        for i in range(len(pred)):
            if pred[i] == 0:
                pred[i] = 1
            else:
                pred[i] = 0
    return pred

In [326]:
data_file = 'clusterFuzzy.csv'
sf = SFrame.read_csv(data_file)

Finished parsing file /Users/hantaoliu/Documents/OD/clusterFuzzy.csv

Parsing completed. Parsed 100 lines in 0.098808 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,float,float,float,float,int,float,str,str,str,str,float,float,float,float,int,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/hantaoliu/Documents/OD/clusterFuzzy.csv

Parsing completed. Parsed 5064 lines in 0.088322 secs.

In [361]:
time_frame

date,end_time,start_time,trip_id
2018-03-06,11:36,10:51,2
2018-03-06,8:32,7:40,3
2018-03-01,11:51,11:03,4
2018-11-04,15:47,15:05,5
2018-11-04,17:21,16:55,6
2018-11-08,15:33,15:05,7
2018-11-08,16:04,15:38,8
2018-11-09,19:26,18:40,9
2018-11-09,20:05,19:38,10


In [360]:
list_trip = []
trip = False
for i in sf:
    start = i['timeStart']
    end = i['timeEnd']
    date = format_transform(i['date'])
    for j in time_frame:
        s = j['start_time']
        e = j['end_time']
        if date == j['date'] and time_difference(s, start) >= 0 and time_difference(end, e) >= 0:
            list_trip.append(j['trip_id'])
            break
    
list_trip
# sf = sf.add_columns(SFrame({'trip_id': list_trip}))

['2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2'

In [275]:
def add_one(list_a, s, e):
    for i in range(s, e):
        list_a[i] += 1
    return list_a

In [544]:
# '''
# Matt method
# '''

# def assign_count(stop_frame, data):
#     list_num = [0 for i in range(len(stop_frame))]
#     list_off = [0 for i in range(len(stop_frame))]
#     list_on = [0 for i in range(len(stop_frame))]
#     for i in data:
#         if i['label'] != 1:
#             continue
#         trip_id = i['trip_id']
#         lat_s = i['start_lat']
#         lon_s = i['start_lon']
#         lat_e = i['end_lat']
#         lon_e = i['end_lon']
#         min_first = 9999999999
#         min_end = 9999999999
#         origin = 0
#         destin = 0
#         for inx, val in enumerate(stop_frame):
#             if trip_id == val['trip_id']:
#                 coords_1 = (lat_s, lon_s)
#                 coords_2 = (val['lat'], val['lon'])
#                 coords_3 = (lat_e, lon_e)
#                 first_distance = geopy.distance.geodesic(coords_1, coords_2).m
#                 end_distance = geopy.distance.geodesic(coords_3, coords_2).m
#                 if first_distance < min_first:
#                     min_first = first_distance
#                     origin = inx
#                 if end_distance < min_end:
#                     min_end = end_distance
#                     destin = inx
#         list_on[origin] += 1
#         list_off[destin] += 1
#         list_num = add_one(list_num, origin, destin)
#     return stop_frame.add_columns(SFrame({'total': list_num, 'on': list_on, 'off': list_off}))
                

In [170]:
# '''
# Rabi et al method
# '''

# def assign_count(stop_frame, data):
#     thresh = 54.86
#     list_num = [0 for i in range(len(stop_frame))]
#     list_off = [0 for i in range(len(stop_frame))]
#     list_on = [0 for i in range(len(stop_frame))]
#     # First look for two stops that the data is belonging to 
#     for i in data:
#         if i['label'] != 1:
#             continue
#         det_s_time = i['timeStart']
#         det_e_time = i['timeEnd']
#         trip_id = i['trip_id']
#         found_board = False
#         found_alight = False
#         lat_s = i['start_lat']
#         lon_s = i['start_lon']
#         lat_e = i['end_lat']
#         lon_e = i['end_lon']
#         first = 0
#         last = 0
#         count = 0
#         for inx, val in enumerate(stop_frame):
#             if trip_id != val['trip_id']:
#                 continue
#             else:
#                 count += 1
#                 time = val['time']
#                 if inx > 0:
#                     lat1 = stop_frame[inx - 1]['lat']
#                     lon1 = stop_frame[inx - 1]['lon']
#                 lat2 = stop_frame[inx]['lat']
#                 lon2 = stop_frame[inx]['lon']
#                 if found_board and found_alight:
#                     break
#                 elif not found_board:
#                     if time_difference(time, det_s_time) < 0:
#                         coords_1 = (lat_s, lon_s)
#                         coords_2 = (lat2,lon2)
#                         first = inx if (geopy.distance.geodesic(coords_1, coords_2).m <= thresh) else inx - 1
#                         found_board = True
#                 else:
#                     if time_difference(time, det_e_time) < 0:
#                         coords_1 = (lat_s, lon_s)
#                         coords_2 = (lat1,lon1)
#                         last = inx - 1 if (geopy.distance.geodesic(coords_1, coords_2).m <= thresh) else inx
#                         found_alight = True
#         if last != 0:
#             list_off[last] += 1
#             list_num = add_one(list_num, first, last)
#         else:
#             list_num = add_one(list_num, first, count)
#         list_on[first] += 1
        
#     return stop_frame.add_columns(SFrame({'on': list_on, 'off': list_off, 'total': list_num}))


In [602]:
'''
This method checks if the MAC is charcterized as passenger
and assign it to every stop which the MAC has been to
'''

def returnMin(time_frame, time, date, trip):
    minimum = 100000000
    index = 0
    count = 0
    for i in time_frame:
        # Compute the distance between two times
        if i['trip_id'] == trip:
            difference = abs(time_difference(i['time'], time))
            if difference < minimum:
                minimum = difference
                index = count
        count+=1
    return index, (time_frame[index]['passenger'] == 0)

def assign_count(stop_frame, data):
    list_num = [0 for i in range(len(stop_frame))]
    list_off = [0 for i in range(len(stop_frame))]
    list_on = [0 for i in range(len(stop_frame))]
    for i in data:
        if i['label'] == 1:
            date = format_transform(i['date'])
            start_time = i['timeStart']
            end_time = i['timeEnd']
            trip = i['trip_id']
            # Compute the stop interval 
            start_index,_ = returnMin(stop_frame, start_time, date, trip)
            end_index,_ = returnMin(stop_frame, end_time, date, trip)

            if end_index != -1:
                list_off[end_index] += 1
                list_num = add_one(list_num, start_index, end_index)
            else:
                end_index = check_last_index(stop_frame['trip_id'], trip)
                list_num = add_one(list_num, start_index, end_index)
            list_on[start_index] += 1
    return stop_frame.add_columns(SFrame({'on': list_on, 'off': list_off, 'total': list_num}))


In [603]:
passen_frame = assign_count(stop_frame, sf)

In [604]:
# myFile = open('passen_frame_Kobe.csv', 'w')  
# with myFile:  
#     writer = csv.writer(myFile)
#     writer.writerows(passen_frame.to_dataframe().values)
passen_frame.export_csv('clusterFuzzy_Original.csv')

In [672]:
passen_frame = SFrame.read_csv('clusterFuzzyPassenger1.csv')

Finished parsing file /Users/hantaoliu/Documents/OD/clusterFuzzyPassenger1.csv

Parsing completed. Parsed 100 lines in 0.031266 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,int,int,str,int,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/hantaoliu/Documents/OD/clusterFuzzyPassenger1.csv

Parsing completed. Parsed 191 lines in 0.010373 secs.

In [530]:
from sklearn import datasets, linear_model
import keras
from sklearn.ensemble import RandomForestRegressor

In [673]:
rr_data = passen_frame

In [484]:
def normalize(v, maxVal):
    largest = v.max()
    if largest == 0: 
        return v
    if largest < maxVal:
        return v / maxVal
    return v / largest

In [682]:
train_h = list()
train_m = list()
train_total = list()
test_total = list()
train_GT = list()
train_mOn = list()
train_hOn = list()
train_mOff = list()
train_hOff = list()
train_on = list()
train_off = list()
train_c_on = list()
train_c_off = list()
test_m = list()
test_h = list()
test_c = list()
test_on = list()
test_off = list()
test_c_on = list()
test_c_off = list()
for i in rr_data:
    if i['trip_id'] == 10:
        test_on.append(int(i['onboard']))
        test_off.append(int(i['offboard']))
        test_h.append(int(i['arrtime'].split()[1].split(':')[0]))
        test_m.append(int(i['arrtime'].split()[1].split(':')[1]))
        test_c_on.append(int(i['on']))
        test_c_off.append(int(i['off']))
        test_total.append(int(i['total']))
        continue
    if i['trip_id'] != 10:
        train_h.append(int(i['arrtime'].split()[1].split(':')[0]))
        train_m.append(int(i['arrtime'].split()[1].split(':')[1]))
        train_total.append(int(i['total']))
        train_GT.append(int(i['passenger']))
    
    if not i['trip_id'] == 6 and not i['trip_id'] == 10:
        train_on.append(int(i['onboard']))
        train_c_on.append(int(i['on']))
        train_hOn.append(int(i['arrtime'].split()[1].split(':')[0]))
        train_mOn.append(int(i['arrtime'].split()[1].split(':')[1]))
    if not i['trip_id'] == 6 and not i['trip_id'] == 8 and not i['trip_id'] == 10:
        train_off.append(int(i['offboard']))
        train_c_off.append(int(i['off']))
        train_hOff.append(int(i['arrtime'].split()[1].split(':')[0]))
        train_mOff.append(int(i['arrtime'].split()[1].split(':')[1]))

rfTrain_total = SFrame({'total': train_total, 'hour': train_h, 'minute': train_m})
rfTest_total = SFrame({'total': test_total, 'hour': test_h, 'minute': test_m})
rfTrain_on = SFrame({'on': train_c_on, 'hour': train_hOn, 'minute': train_mOn})
rfTest_on = SFrame({'on': test_c_on, 'hour': test_h, 'minute': test_m})
rfTrain_off = SFrame({'off': train_c_off, 'hour': train_hOff, 'minute': train_mOff})
rfTest_off = SFrame({'off': test_c_off, 'hour': test_h, 'minute': test_m})

In [683]:
features = ['on', 'hour', 'minute']
for f in features:
    maxVal = max(rfTrain_on[f])
    rfTrain_on[f] = normalize(rfTrain_on[f], maxVal)
    rfTest_on[f] = normalize(rfTest_on[f], maxVal)

features = ['off', 'hour', 'minute']
for f in features:
    maxVal = max(rfTrain_off[f])
    rfTrain_off[f] = normalize(rfTrain_off[f], maxVal)
    rfTest_off[f] = normalize(rfTest_off[f], maxVal)
    

In [684]:
rfTrain_on = rfTrain_on.to_numpy()
rfTest_on = rfTest_on.to_numpy()
rfTrain_off = rfTrain_off.to_numpy()
rfTest_off = rfTest_off.to_numpy()
rfTrain_total = rfTrain_total.to_numpy()
rfTest_total = rfTest_total.to_numpy()

In [685]:
RFReg = RandomForestRegressor(random_state=0, n_estimators=100)
RFReg.fit(rfTrain_on, train_on)
predict_on = np.around(RFReg.predict(rfTest_on))
predict_on

array([4., 0., 2., 1., 1., 1., 3., 3., 2., 1., 0., 2., 3., 0., 0.])

In [686]:
RFReg = RandomForestRegressor(random_state=0, n_estimators=100)
RFReg.fit(rfTrain_off, train_off)
predict_off = np.around(RFReg.predict(rfTest_off))
predict_off

array([1., 1., 2., 4., 2., 1., 1., 1., 2., 3., 1., 2., 3., 4., 4.])

In [687]:
RFReg = RandomForestRegressor(random_state=0, n_estimators=100)
RFReg.fit(rfTrain_total, train_GT)
predict_total = np.around(RFReg.predict(rfTest_total))
predict_total

array([ 4.,  6.,  5.,  5.,  5.,  5.,  5.,  5., 12., 13., 11., 11., 12.,
       17., 17.])

In [688]:
with open("predict_total.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(map(lambda x: [x], predict_total))

In [527]:
import math
def counter_cosine_similarity(c1, c2):
    dotprod = sum(c1[k] * c2[k] for k in range(len(c1)))
    magA = math.sqrt(sum(c1[k]**2 for k in range(len(c1))))
    magB = math.sqrt(sum(c2[k]**2 for k in range(len(c1))))
    return dotprod / (magA * magB)

In [607]:
trips = []
score = dict()
totalOn = 0
totalOff = 0
for i in time_frame:
    onGT = list()
    offGT = list()
    on = list()
    off = list()
    for j in rr_data:
        if i['trip_id'] == str(j['trip_id']):
            onGT.append(j['onboard'])
            offGT.append(j['offboard'])
            on.append(j['on'])
            off.append(j['off'])
    score[i['trip_id'] + 'on'] = counter_cosine_similarity(onGT, on)
    score[i['trip_id'] + 'off'] = counter_cosine_similarity(offGT, off)
    totalOn += counter_cosine_similarity(onGT, on)
    totalOff += counter_cosine_similarity(offGT, off)
    
avgOn = totalOn / len(time_frame)
avgOff = totalOff / len(time_frame)
score

{'10off': 0.914991421995628,
 '10on': 0.5847053462046861,
 '2off': 0.5687401397252118,
 '2on': 0.5951740664074515,
 '3off': 0.6538318840471801,
 '3on': 0.7650386871041243,
 '4off': 0.5361543068509792,
 '4on': 0.594823788100279,
 '5off': 0.5115771579541613,
 '5on': 0.5043359185638772,
 '6off': 0.4522670168666455,
 '6on': 0.4824752523663836,
 '7off': 0.5157106231293966,
 '7on': 0.6266283136583128,
 '8off': 0.24253562503633297,
 '8on': 0.5319951765989315,
 '9off': 0.620453387463585,
 '9on': 0.8409178658720821}

In [680]:
def returnMin(time_frame, time):
    minimum = 100000000
    index = 0
    count = 0
    for i in time_frame:
        # Compute the distance between two times
        difference = abs(time_difference(i['time'], time))
        if difference < minimum:
            minimum = difference
            index = count
        count+=1
    return index

matrix = [[0 for x in range(15)] for y in range(15)]
for i in sf:
    if i['trip_id'] == str(10) and i['label'] == 1:
        start_time = i['timeStart']
        end_time = i['timeEnd']
        start = returnMin(stop_frame[176:191], start_time)
        end = returnMin(stop_frame[176:191], end_time)

        if start == end and end != 14:
            end += 1
        elif start == end and end == 14:
            start -= 1
        matrix[start][end] += 1

In [681]:
with open("trip_10V3.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(matrix)

In [459]:
test_on = [1,2,0,1,1,1,1,2,4,4,1,4,2,0,2,1,1,2,2,0,2,1,0,3,0,0,0]
test_off = [0,0,0,0,0,0,0,1,4,5,1,1,2,2,0,2,2,1,2,2,0,3,2,2,5,0,1]

In [402]:
test_m = list()
test_h = list()
test_gt = list()
for i in rr_data:
    if i['trip_id'] == 5:
        test_h.append(int(i['arrtime'].split()[1].split(':')[0]))
        test_m.append(int(i['arrtime'].split()[1].split(':')[1]))
        test_gt.append(int(i['passenger']))
        continue

In [460]:
rfTestOn = []
rfTestOff = []

for i in range(len(test_on)):
    list_test_on = []
    list_test_off = []
    list_test_on = [int(test_h[i])/max(test_h)] + [int(test_m[i])/max(test_m)] + [int(test_on[i])/max(test_on)]
    list_test_off = [int(test_h[i])/max(test_h)] + [int(test_m[i])/max(test_m)] + [int(test_off[i])/max(test_on)]
    rfTestOn.append(list_test_on)
    rfTestOff.append(list_test_off)

In [461]:
predict_on = np.around(RFReg.predict(rfTestOn))
predict_on

array([10., 13., 13.,  9.,  9., 12., 19., 11., 28., 27., 17., 27., 14.,
       16., 15., 14., 14., 15., 15.,  4., 14., 17., 14., 34., 26., 21.,
       21.])

In [689]:
with open("predict_on.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(map(lambda x: [x], predict_on))

In [690]:
with open("predict_off.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(map(lambda x: [x], predict_off))

In [115]:
'''
This method plot the passenger number for each trip
'''
def plot_result(frame, model, title):
    count = 0
    num = 0
    trip_list = [i for i in range(len(time_frame))]
    x = [[] for i in range(len(time_frame))]
    passen = [[] for i in range(len(time_frame))]
    count = [[] for i in range(len(time_frame))]
    pred = [[] for i in range(len(time_frame))]
    stop_id = [[] for i in range(len(time_frame))]
    prediction = model.predict(frame)
    temp = SFrame({'prediction': prediction})
    frame = frame.add_columns(temp)
    for element in frame:
        index = int(element['trip_id']) - int(frame[0]['trip_id'])
        x[index].append(num)
        num += 1
        stop_id[index].append(element['stop_id'])
        passen[index].append(element['passenger'])
        count[index].append(element['count'])
        pred[index].append(element['prediction'])

    for i in range(len(trip_list)):
        plt.figure(i)
        plt.xticks(x[i], stop_id[i])
        plt.plot(x[i], passen[i], 'r-', label = 'Passenger')
        plt.plot(x[i], count[i], 'b-', label = 'MAC Count')
        plt.plot(x[i], pred[i], 'g-', label = 'Prediction')
        plt.xlabel('Stops')
        plt.ylabel('Numbers')
        plt.title(title)
        plt.legend(loc='upper left')
        plt.show()

In [124]:
'''
This method finds the least MSE model and
training testing combination
'''
def find_best_model(model, times):
    lowest_MSE = 1000000
    final_result = []
    target_train = []
    target_test = []
    # Normalized the features
    features = ['count', 'hour', 'minute', 'route']
    for f in features:
        print(f)
        model[f] = normalize(model[f])
    for i in range(times):
        train_data, test_data = model.random_split(.8, seed=1)
        psen_model = tc.linear_regression.create(train_data, target='passenger', 
                                          features=['count', 'hour', 'minute', 'weekday', 
                                                    'last_stop', 'route'], verbose=False)
        result = psen_model.predict(test_data)
        score = mape(test_data['passenger'], result)
        if score < lowest_MSE:
            final_model = psen_model
            lowest_MSE = score
            target_train = train_data
            target_test = test_data
            final_result = result
    print(result)
    print(target_test['passenger'])
    evaluate(final_result, test_data['passenger'])
    return final_model

In [127]:
'''
This method generate OD matrice
'''
def OD_Matrix(frame, time_frame):
    od_matrix = []
    for k in range(len(time_frame)):
        length = len(frame[frame['trip_id'] == time_frame[k]['trip_id']])
        end = check_last_index(frame['trip_id'], time_frame[k]['trip_id'])

        list_mac = []
        prev_mac = []
        single_trip_matrix = [[0 for i in range(length)] for j in range(length)]
        for i in range(end + 1 - length, end + 1):
            list_mac = frame[i]['mac']
            on_board = list_difference(list_mac, prev_mac)
            prev_mac = list_mac
            for j in range(i + 1, end + 1):
                next_stop_mac = frame[j]['mac']
                off_board = list_difference(on_board, next_stop_mac)
                on_board = list_difference(on_board, off_board)
                single_trip_matrix[i-(end+1-length)][j-(end+1-length)] += len(off_board)
        od_matrix.append(single_trip_matrix)
                
    return od_matrix

In [182]:
'''
This step generate all the different OD matrix
according to different outcome from all the clusterings
'''
# Fuzzy
Fuzzy_OD_matrix = OD_Matrix(passen_frame_Fuzzy, time_frame)
# Gaussian
Gaussian_OD_matrix = OD_Matrix(passen_frame_Gaussian, time_frame)
# PCM
PCM_OD_matrix = OD_Matrix(passen_frame_PCM, time_frame)
# SOFM
SOFM_OD_matrix = OD_Matrix(passen_frame_SOFM, time_frame)
# BGM
BGM_OD_matrix = OD_Matrix(passen_frame_BGM, time_frame)
# overall_Kmeans_list = setNone(overall_Kmeans_list)
# info = Table(rows=overall_Gaussian_list)
# write(info, output='OD_Fuzzy.csv', format='csv')

In [158]:
'''
This method extract hour, minute, mac counts, and other features
from given frame and construct a frame that allows the regression 
model to do the predicting
'''
def assignAttribute(time_frame, frame, matrix):
    count = []
    hour = []
    minute = []
    lastStop = []
    Weekday = []
    trips = []
    for i in range(len(matrix)):
        trip = time_frame[i]['trip_id']
        for j in range(len(matrix[i])):     
            for k in range(len(matrix[i])):
                count.append(matrix[i][j][k])
                trips.append(trip)
            for item in frame:
                if item['trip_id'] == trip:
                    hour.append(item['hour'])
                    minute.append(item['minute'])
                    lastStop.append(item['last_stop'])
                    Weekday.append(item['weekday'])
    return SFrame({'tirp': trips, 'count': count, 'hour':hour, 'minute': minute, 
                   'last_stop':lastStop, 'weekday':Weekday})
                

In [183]:
'''
This step create different frame from different clustering
'''
Fuzzy_OD_data = assignAttribute(time_frame, passen_frame_Fuzzy, Fuzzy_OD_matrix)
BGM_OD_data = assignAttribute(time_frame, passen_frame_BGM, BGM_OD_matrix)
SOFM_OD_data = assignAttribute(time_frame, passen_frame_SOFM, SOFM_OD_matrix)
PCM_OD_data = assignAttribute(time_frame, passen_frame_PCM, PCM_OD_matrix)
Gaussian_OD_data = assignAttribute(time_frame, passen_frame_Gaussian, Gaussian_OD_matrix)

In [184]:
'''
This method makes prediction based on the trained model
'''
def make_predict(OD_data, model):
    return model.predict(OD_data)

In [192]:
'''
This method set zero if they had zero count on mac
'''
def setZero(OD_data, result):
    result_new = []
    for i in range(len(OD_data)):
        if OD_data[i]['count'] == 0:
            result_new.append(0)
        else:
            result_new.append(result[i])
    return result_new

In [198]:
'''
This step makes the prediction
'''
result_Fuzzy = make_predict(Fuzzy_OD_data, train_Fuzzy_model)
result_Fuzzy = setZero(Fuzzy_OD_data, result_Fuzzy)

result_BGM = make_predict(BGM_OD_data, train_BGM_model)
result_BGM = setZero(BGM_OD_data, result_BGM)

result_SOFM = make_predict(SOFM_OD_data, train_SOFM_model)
result_SOFM = setZero(SOFM_OD_data, result_SOFM)

result_Gaussian = make_predict(Gaussian_OD_data, train_Gaussian_model)
result_Gaussian = setZero(Gaussian_OD_data, result_Gaussian)

result_PCM = make_predict(PCM_OD_data, train_PCM_model)
result_PCM = setZero(PCM_OD_data, result_PCM)